In [53]:
#Dependencies
import requests
import json
import pandas as pd

url = "https://www.fantasyfootballdatapros.com/api/players/"
years = list(range(2010, 2020))

player_name = []
position = []
season = []
games_played = []
fumbles = []
interceptions = []
passing_yds = []
receptions = []
receiving_yds = []
rushing_yds = []
passing_tds = []
rushing_tds = []
receiving_tds = []
team = []

#fantasy = requests.get(url + '2019/all').json()
#print(json.dumps(fantasy, indent=4, sort_keys=True))

for year in years:
    query_url = url + str(year) + '/all'
    fantasy = requests.get(query_url).json()
    number_of_players = len(fantasy)
    
    for x in range(number_of_players):
        player_name.append(fantasy[x]['player_name'])
        team.append(fantasy[x]['team'])
        position.append(fantasy[x]['position'])
        season.append(year)
        games_played.append(fantasy[x]['games_played'])
        fumbles.append(fantasy[x]['fumbles_lost'])
        interceptions.append(fantasy[x]['stats']['passing']['int'])
        passing_yds.append(fantasy[x]['stats']['passing']['passing_yds'])
        passing_tds.append(fantasy[x]['stats']['passing']['passing_td'])
        rushing_yds.append(fantasy[x]['stats']['rushing']['rushing_yds'])
        rushing_tds.append(fantasy[x]['stats']['rushing']['rushing_td'])
        receptions.append(fantasy[x]['stats']['receiving']['receptions'])
        receiving_yds.append(fantasy[x]['stats']['receiving']['receiving_yds'])
        receiving_tds.append(fantasy[x]['stats']['receiving']['receiving_td'])
    
player_dict = {
    'Player Name' : player_name,
    'Team': team,
    'Position': position,
    'Season': season,
    'Games Played': games_played,
    'Fumbles': fumbles,
    'Interceptions': interceptions,
    'Passing Yards': passing_yds,
    'Passing TDs':passing_tds,
    'Rushing Yards':rushing_yds,
    'Rushing TDs':rushing_tds,
    'Receptions':receptions,
    'Receiving Yards':receiving_yds,
    'Receiving TDs':receiving_tds
}
    
players = pd.DataFrame(player_dict)

players['Half PPR Points Season Total'] = (
    (players['Passing Yards']*0.25) + (players['Passing TDs'] * 4) - (players['Interceptions'] * 2)+
    (players['Rushing Yards']*.1) + (players['Rushing TDs'] * 6) - (players['Fumbles'] * 2) + 
    (players['Receptions'] * 0.5) + (players['Receiving Yards'] * .1) + (players['Receiving TDs'] * 6)
    )





In [61]:
players.head(15)

qbs = players[players['Position'] == 'QB']
rbs = players[players['Position'] == 'RB']
wrs = players[players['Position'] == 'WR']
tes = players[players['Position'] == 'TE']

tes.head()

,Player Name,Team,Position,Season,Games Played,Fumbles,Interceptions,Passing Yards,Passing TDs,Rushing Yards,Rushing TDs,Receptions,Receiving Yards,Receiving TDs,Half PPR Points Season Total
23,Jason Witten,DAL,TE,2010,16.0,1.0,0.0,0.0,0.0,0.0,0.0,94.0,1002.0,9.0,199.2
35,Antonio Gates,SDG,TE,2010,10.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,782.0,10.0,163.2
38,Vernon Davis,SFO,TE,2010,16.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,914.0,7.0,161.4
41,Marcedes Lewis,JAX,TE,2010,16.0,2.0,0.0,0.0,0.0,0.0,0.0,58.0,700.0,10.0,155.0
52,Rob Gronkowski,NWE,TE,2010,16.0,1.0,0.0,0.0,0.0,0.0,0.0,42.0,546.0,10.0,133.6


In [ ]:
#Rookie Lists
#QB
qb2019 = ["Kyler Murray", "Daniel Jones", "Dwayne Haskins", "Drew Lock", "Will Grier"]
qb2018 = ["Baker Mayfield", "Sam Darnold", "Josh Allen", "Josh Rosen", "Lamar Jackson"]
qb2017 = ["Mitchell Trubisky", "Patrick Mahomes", "Deshaun Watson", "DeShone Kizer", "Davis Webb"]
qb2016 = ["Jared Goff", "Carson Wentz", "Paxton Lynch", "Christian Hackenberg", "Jacoby Brissett"]
qb2015 = ["Jameis Winston", "Marcus Mariota", "Garrett Grayson", "Sean Mannion", "Bryce Petty"]

#RB
rb2019 = [Josh Jacobs, Miles Sanders, Darrell Henderson, David Montgomery, Devin Singletary]
rb2018 = [Saquon Barkley]
rb2017 = 
rb2016 = 
rb2015 = 